In [ ]:
# --- 288.24240469932556 seconds ---

import os
import click
import gym
import pybullet_envs
import neat
import pickle
import random
import signal
import matplotlib.pyplot as plt
import numpy as np
import torch
import time

from pytorch_neat.recurrent_net import RecurrentNet
from pytorch_neat.multi_env_eval import MultiEnvEvaluator

num_cores = 8

config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'data/config-humanoid')
p = neat.Population(config)
p.add_reporter(neat.StdOutReporter(True)) 
p.add_reporter(neat.StatisticsReporter())
p.add_reporter(neat.Checkpointer(1, None, "saves/robotic_pytorch_checkpoint_"))

In [ ]:
max_env_steps = 10

def make_env():
    return gym.make('AntBulletEnv-v0')


def make_net(genome, config, bs):
    return RecurrentNet.create(genome, config, bs)


def activate_net(net, states):
    outputs = net.activate(states).numpy()
    return outputs

def eval_genomes(genome, config):
    evaluator = MultiEnvEvaluator(make_net, activate_net, make_env=make_env, max_env_steps=max_env_steps)
    fitn = evaluator.eval_genome(genome, config)
    return fitn

def run(n_generations):
    # Load the config file, which is assumed to live in
    # the same directory as this script.
    
    pe = neat.ParallelEvaluator(num_cores, eval_genomes)

    winner = p.run(pe.evaluate, n_generations)

    with open('winnerPara.pkl', 'wb') as output:
        pickle.dump(winner, output, 1)
        
    print('\nFinished executing')

In [ ]:
start_time = time.time()
run(1)
print("--- %s seconds ---" % (time.time() - start_time))

In [31]:
with open('winnerPara.pkl','rb') as readWinner:
    g = pickle.load(readWinner)
    env = gym.make('AntBulletEnv-v0')
    while True:
        try:
            observation = env.reset()
            net = RecurrentNet.create(g, config)
            reward = 0

            for t in range(250):

                time.sleep(0.01)
                env.render()

                # interact with NN
                
                #changed_ob = [np.array(observation)]
                changed_ob = torch.tensor([observation])
            
                action = net.activate(changed_ob)
                observation, reward, done, info = env.step(action)
                
                print('Blaaaaaaaaaaaaaaaaaaaaaaa')

                if done:
                    break

        except Exception as ex:
            print(ex)
            time.sleep(0.2)

only one element tensors can be converted to Python scalars
only one element tensors can be converted to Python scalars
only one element tensors can be converted to Python scalars
only one element tensors can be converted to Python scalars
only one element tensors can be converted to Python scalars
only one element tensors can be converted to Python scalars
only one element tensors can be converted to Python scalars
only one element tensors can be converted to Python scalars
only one element tensors can be converted to Python scalars
only one element tensors can be converted to Python scalars
only one element tensors can be converted to Python scalars
only one element tensors can be converted to Python scalars
only one element tensors can be converted to Python scalars
only one element tensors can be converted to Python scalars
only one element tensors can be converted to Python scalars
only one element tensors can be converted to Python scalars
only one element tensors can be converte

KeyboardInterrupt: 

In [ ]:
# --- 295.45882081985474 seconds ---

import multiprocessing
import os

import click
import neat
import gym
import torch
import numpy as np

import time

from pytorch_neat.recurrent_net import RecurrentNet
from pytorch_neat.multi_env_eval import MultiEnvEvaluator

batch_size = 4
DEBUG = True

In [ ]:
def make_env():
    return gym.make('AntBulletEnv-v0')


def make_net(genome, config, bs):
    return RecurrentNet.create(genome, config, bs)


def activate_net(net, states):
    outputs = net.activate(states).numpy()
    return outputs

def run(n_generations, n_processes):
    # Load the config file, which is assumed to live in
    # the same directory as this script.
    config_path = "data/config-humanoid"
    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        config_path,
    )
    
    evaluator = MultiEnvEvaluator(make_net, activate_net, make_env=make_env, max_env_steps=1000)
    
    pop = neat.Population(config)
    stats = neat.StatisticsReporter()
    pop.add_reporter(stats)
    reporter = neat.StdOutReporter(True)
    pop.add_reporter(reporter)

    if n_processes > 1:
        pool = multiprocessing.Pool(processes=n_processes)

        def eval_genomes(genomes, config):
            fitnesses = pool.starmap(
                evaluator.eval_genome, ((genome, config) for _, genome in genomes)
            )
            for (_, genome), fitness in zip(genomes, fitnesses):
                genome.fitness = fitness

    else:

        def eval_genomes(genomes, config):
            for i, (_, genome) in enumerate(genomes):
                try:
                    genome.fitness = evaluator.eval_genome(
                        genome, config, debug=DEBUG and i % 100 == 0
                    )
                except Exception as e:
                    print(genome)
                    raise e

    

    winner = pop.run(eval_genomes, n_generations)

    print(winner)
    final_performance = evaluator.eval_genome(winner, config)
    print("Final performance: {}".format(final_performance))
    generations = reporter.generation + 1
    return generations

In [ ]:
start_time = time.time()
gens = run(5,8)
print("--- %s seconds ---" % (time.time() - start_time))
print('gens: '+str(gens))

In [25]:
x = [1,2,3]
x = torch.tensor(x)

In [26]:
print('type: '+str(type(x)))
print('solo: '+str(x))
y = x.numpy()
print('type: '+str(type(y)))
print('solo: '+str(y))

type: <class 'torch.Tensor'>
solo: tensor([1, 2, 3])
type: <class 'numpy.ndarray'>
solo: [1 2 3]
